In [3]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas  as pd
import numpy as np

In [4]:
### load the trained model , scaler pickle, onehot
model=load_model('model.h5')

## load the encoder and scaler
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender=pickle.load(file)

with open('onehot_encoder_geo.pkl','rb') as file:
    label_encoder_geo=pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)

In [5]:
# Example input data
input_data={
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

In [6]:
# one_hot_encode 'Geography'
geo_encoded=label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\piyus\anaconda3\envs\my_ann_env\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [7]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [8]:
## Encode the categorical variables
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [9]:
### concatination one hot encoded
input_df=pd.concat([input_df.drop(['Geography'],axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [10]:
### scaling the input Data
input_scaled=scaler.transform(input_df)

In [11]:
## Predict churn
prediction=model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step


array([[0.04649232]], dtype=float32)

In [12]:
# finding the probablity of the churn
prediction_prob=prediction[0][0]
prediction_prob

np.float32(0.04649232)

In [13]:
if prediction_prob>0.5:
    print('The customer is likely to churn')
else:
    print('the customer is not liklely to churn')

the customer is not liklely to churn
